# NLP with RNNs and Attention

## Using Character RNNs

Character RNNs or char-RNNs are RNNs that try to predict the next character or word in a sentence allowing them to generate text

### Example of Generating Text

#### Data Prep

In [1]:
import numpy as np
import tensorflow as tf

2025-04-05 22:35:23.759389: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-05 22:35:23.940396: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743910524.008000    4761 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743910524.027050    4761 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-05 22:35:24.208824: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
shakespeare_url = 'https://homl.info/shakespeare'
filepath = tf.keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [3]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [4]:
## Encode text
text_vec_layer = tf.keras.layers.TextVectorization(split='character', standardize='lower')
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text][0])

I0000 00:00:1743910526.624335    4761 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9679 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


In [5]:
encoded -= 2  ### Drop tokens 0 (pad) and 1 (unknown) which we will not use
n_tokens = text_vec_layer.vocabulary_size() - 2 ### Number of distinct chars 
dataset_size = len(encoded) ### Total number of chars

In [6]:
### Convert to sequences so the model can guess the next character
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(buffer_size=100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [7]:
length = 100
tf.random.set_seed(42)

train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True,
                      seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

#### Building Char-RNN Model

In [8]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.Dense(n_tokens, activation='softmax')
# ])
# model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam',
#              metrics=['accuracy'])
# model_ckpt = tf.keras.callbacks.ModelCheckpoint(
#     'my_shakespeare_model.keras', monitor='val_accuracy', save_best_only=True
# )
# history = model.fit(train_set, validation_data=valid_set, epochs=10,
#                    callbacks=[model_ckpt])

In [9]:
model = tf.keras.models.load_model('my_shakespeare_model.keras')

In [10]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2), ### NO pad or <UNK> tokens
    model
])

In [11]:
# Test Model
test_input = tf.constant(["To be or not to b"])
y_proba = shakespeare_model.predict(test_input)[0, -1]
y_pred = tf.argmax(y_proba) ## Choose the most probable character ID
text_vec_layer.get_vocabulary()[y_pred + 2]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 619ms/step


I0000 00:00:1743910528.900190    4828 cuda_dnn.cc:529] Loaded cuDNN version 90300


'e'

### Generating Fake SS Text

In [12]:
log_probas = tf.math.log([[0.5, 0.4, 0.1]])
tf.random.set_seed(42)
tf.random.categorical(log_probas, num_samples=8)

<tf.Tensor: shape=(1, 8), dtype=int64, numpy=array([[0, 0, 1, 1, 1, 0, 0, 0]])>

In [38]:
def next_char(text, temperature=1):
    text = tf.constant([text])
    y_proba = shakespeare_model.predict(text, verbose=0)[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0,0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [39]:
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [40]:
print(extend_text('To be or not to be', temperature=0.01))

To be or not to be the duke?

provost:
i have some services have som


In [41]:
print(extend_text('To be or not to be', temperature=1))

To be or not to be's dangerers,
and she had been to die by the sun t


In [42]:
print(extend_text('To be or not to be', temperature=100))

To be or not to beas'xosq;!jorar:p?i.ies;astsz-;jbrpi!de$s?cp;??
:sb


### Stateful RNNs